In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms

!pip install kornia
from kornia import augmentation as K
from kornia.augmentation import AugmentationSequential
from torch.utils.data import random_split

import numpy as np

import os
import time
from pathlib import Path
import pickle

import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import Subset


DEBUG = False
VERSION = "biomass"
env = 'kaggle' # 'kaggle' or 'colab'

if env == 'colab':
    from google.colab import drive
    drive.mount('/content/drive')
    base_dir = Path('/content/drive/MyDrive/dl_pj')    
elif env == 'kaggle':
    base_dir = Path('/kaggle/working/')

checkpoint_dir = base_dir / 'checkpoints'
checkpoint_dir.mkdir(parents=True, exist_ok=True)
training_stats_dir = base_dir / 'stats'
training_stats_dir.mkdir(parents=True, exist_ok=True)

SAVE_PATH_BEST_MODEL = checkpoint_dir / f"baseline_max_mse_{VERSION}.pth"
SAVE_PATH_STATS = training_stats_dir / f'baseline_{VERSION}_stats.pkl'

In [2]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,kernel_size=1,stride=stride,bias=False),
                nn.BatchNorm2d(planes)
            )


    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out



class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layer = []
        for s in strides:
            layer.append(block(self.in_planes, planes, s))
            self.in_planes = planes
        return nn.Sequential(*layer)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.maxpool1(out)
        
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        out = F.adaptive_avg_pool2d(out, (1,1))
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=5)


In [3]:
def get_model_summary(model):
    num_params = sum(p.numel() for p in model.parameters())
    total_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
    size_mb = total_bytes / (1024 ** 2)
    return num_params, size_mb

total_params, model_size_mb = get_model_summary(ResNet18())
print(f"Total Parameters: {total_params:,}")
print(f"Model Size: {model_size_mb:.2f} MB")

Total Parameters: 11,179,077
Model Size: 42.64 MB


In [4]:
def calculate_mse(model, dataloader, device):
    model.eval() 
    total_mse = 0
    total_samples = 0
    criterion = nn.MSELoss(reduction='sum')
    with torch.no_grad():
        for images, targets in dataloader:
            images = images.to(device)
            targets = targets.to(device)
            
            outputs = model(images)
            
            total_samples += targets.size(0)
            total_mse += criterion(outputs, targets).item()

    
    return total_mse / total_samples


In [5]:
TARGETS_FACTOR = 1/2000
class BiomassDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, test_mode=False):
        self.img_dir = img_dir
        self.transform = transform
        self.test_mode = test_mode
        if not self.test_mode:
            df = pd.read_csv(csv_file)
            df[['image_id', 'tmp']] = df['sample_id'].str.split("__", expand=True)
            self.data = df.pivot(index='image_id', columns='target_name', values='target')
            self.target_cols = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
            self.data = self.data[self.target_cols].reset_index()
            self.image_ids = self.data['image_id'].values
            self.targets = self.data[self.target_cols].values.astype('float32') * TARGETS_FACTOR
        else:
            self.image_ids = [f.split('.')[0] for f in os.listdir(img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id  = self.image_ids[idx]
    
        image_name = os.path.join(self.img_dir, f"{image_id}.jpg")
    
        image = Image.open(image_name).convert('RGB')
    
        if self.transform:
            image = self.transform(image)
        
        if self.test_mode:
            return image, image_id
    
        target = torch.tensor(self.targets[idx])
        
        return image, target

In [6]:
import torchvision.transforms as T

mean = torch.tensor([0.4914, 0.4822, 0.4465])
std = torch.tensor([0.2023, 0.1994, 0.2010])

train_transform = T.Compose([
    T.RandomCrop((448,448)),
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std)
])

val_transform = T.Compose([
    T.CenterCrop((448,448)),
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std)
])


train_dataset_full = BiomassDataset(
    csv_file='/kaggle/input/csiro-biomass/train.csv',
    img_dir='/kaggle/input/csiro-biomass/train',
    transform=train_transform
)

val_dataset_full = BiomassDataset(
    csv_file='/kaggle/input/csiro-biomass/train.csv',
    img_dir='/kaggle/input/csiro-biomass/train',
    transform=val_transform
)

dataset_size = len(train_dataset_full)
train_len = int(0.8 * dataset_size)
val_len = dataset_size - train_len
generator = torch.Generator().manual_seed(42)
train_subset_tmp, val_subset_tmp = random_split(
    train_dataset_full, [train_len, val_len], generator=generator
)
train_idx = train_subset_tmp.indices
val_idx = val_subset_tmp.indices

trainset = Subset(train_dataset_full, train_idx)
valset = Subset(val_dataset_full, val_idx)

testset = BiomassDataset(
    csv_file='/kaggle/input/csiro-biomass/test.csv',
    img_dir='/kaggle/input/csiro-biomass/test',
    transform=val_transform,
    test_mode=True
)

In [7]:
# Hyperparamters
batch_size = 32

lr = 1e-4
momentum = 0.9
weight_decay = 5e-4

T_max = 200

n_epochs = 1 if DEBUG else 200

print_progress_every = 1
val_mse_storing_threshold = 1


In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=2)
model = ResNet18().to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max)

stats = {
    'total_training_time': 0,
    'loss': [],
    'time_per_epoch': [],
    'total_time_per_epoch': [],
    'val_mse': [],
    'min_val_mse': float('inf'),
    'allocated_memory': [], # Memory currently used by Tensors
    'reserved_memory': [], # Memory held by the PyTorch caching allocator
}

start_time = time.time()
for epoch in range(n_epochs):
    model.train()
    iteration_losses = []
    epoch_start_time = time.time()
    for inputs, targets in trainloader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)

        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        iteration_losses.append(loss.item())

    scheduler.step()
    epoch_end_time = time.time()

    model.eval()
    val_mse = calculate_mse(model, valloader, device)

    # Track stats
    if (epoch % 1) == 0:
        stats['loss'].append(
            np.mean(iteration_losses)
        )
        stats['val_mse'].append(
            val_mse
        )
        stats['allocated_memory'].append(torch.cuda.memory_allocated())
        stats['reserved_memory'].append(torch.cuda.memory_reserved())
        stats['time_per_epoch'].append(epoch_end_time - epoch_start_time)
        stats['total_time_per_epoch'].append(time.time() - start_time)

    # Store best model
    if (val_mse < stats['min_val_mse']):
        if (val_mse < val_mse_storing_threshold):
            stats['min_val_mse'] = val_mse
            print('==> Saving model ...')
            state = {
                'net': model.state_dict(),
                'epoch': epoch,
                'mse':val_mse
            }
            torch.save(state, SAVE_PATH_BEST_MODEL)

    if DEBUG:
        print("DEBUG: One epoch time:", time.time() - start_time)
        print('DEBUG: ==> Saving model ...')
        state = {
            'net': model.state_dict(),
            'epoch': epoch,
            'mse':val_mse
        }
        torch.save(state, SAVE_PATH_BEST_MODEL)
        
        
    # Print progress
    if (epoch % print_progress_every) == 0:
        print(f"Epoch {epoch} Train loss {stats['loss'][-1]:.3f} Val mse {stats['val_mse'][-1]:.3f}")



==> Saving model ...
Epoch 0 Train loss 0.160 Val mse 0.052
Epoch 1 Train loss 0.032 Val mse 0.104
Epoch 2 Train loss 0.013 Val mse 0.098
==> Saving model ...
Epoch 3 Train loss 0.016 Val mse 0.047
==> Saving model ...
Epoch 4 Train loss 0.009 Val mse 0.030
Epoch 5 Train loss 0.007 Val mse 0.034
Epoch 6 Train loss 0.007 Val mse 0.034
Epoch 7 Train loss 0.006 Val mse 0.033
Epoch 8 Train loss 0.006 Val mse 0.033
Epoch 9 Train loss 0.007 Val mse 0.033
Epoch 10 Train loss 0.006 Val mse 0.033
Epoch 11 Train loss 0.007 Val mse 0.034
Epoch 12 Train loss 0.006 Val mse 0.034
Epoch 13 Train loss 0.006 Val mse 0.033
Epoch 14 Train loss 0.006 Val mse 0.033
Epoch 15 Train loss 0.006 Val mse 0.033
Epoch 16 Train loss 0.006 Val mse 0.033
Epoch 17 Train loss 0.006 Val mse 0.032
Epoch 18 Train loss 0.006 Val mse 0.032
Epoch 19 Train loss 0.006 Val mse 0.032
Epoch 20 Train loss 0.006 Val mse 0.032
Epoch 21 Train loss 0.006 Val mse 0.032
Epoch 22 Train loss 0.007 Val mse 0.032
Epoch 23 Train loss 0.006 V

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = ResNet18().to(device)
checkpoint = torch.load(SAVE_PATH_BEST_MODEL, map_location=device)
model.load_state_dict(checkpoint['net'])
model.eval()
print("Final val mse is ", calculate_mse(model, valloader, device))
# No real test-set implemented yet, printing val instead


# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)
# print(f'Final test accuracy is: {calculate_accuracy(model, testloader, device):.3f}')

Final val mse is  0.02735179290175438


In [10]:
with open(SAVE_PATH_STATS, 'wb') as file:
    pickle.dump(stats, file)